In [ ]:
!pip install streamlit faiss-cpu langchain sentence-transformers huggingface-hub transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 907.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
# app.py
import streamlit as st
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

st.title("📚 RAG بدون OpenAI - نموذج محلي")

# 1. تحميل الملف
uploaded_file = st.file_uploader("ارفع ملف PDF", type="pdf")

if uploaded_file:
    st.success("تم رفع الملف بنجاح!")

    # 2. استخراج النصوص وتجزئتها
    loader = PyPDFLoader(uploaded_file.name)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    # 3. التضمين Embeddings
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    db = FAISS.from_documents(texts, embeddings)

    # 4. استعلام المستخدم
    query = st.text_input("ما سؤالك حول هذا الكتاب؟")

    if query:
        # استرجاع السياق
        docs = db.similarity_search(query, k=3)
        context = "\n\n".join([doc.page_content for doc in docs])

        # 5. النموذج المحلي
        model_name = "OpenGVLab/InternVL3-1B-Instruct"
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True,
            torch_dtype=torch.float32,
            device_map="auto"
        )

        prompt = f"""أجب على السؤال بناءً على النص التالي:\n\n{context}\n\nالسؤال: {query}\nالإجابة:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        output = model.generate(**inputs, max_new_tokens=200)
        answer = tokenizer.decode(output[0], skip_special_tokens=True)

        st.markdown("### ✅ الإجابة:")
        st.write(answer.split("الإجابة:")[-1].strip())


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
import threading
import time

ngrok_token = "XXXX"
ngrok.set_auth_token(ngrok_token)

# Start ngrok in a separate thread to avoid blocking
def start_ngrok():
    public_url = ngrok.connect(8501).public_url
    print(f"🚀 Ngrok Tunnel Open: {public_url}")

ngrok_thread = threading.Thread(target=start_ngrok)
ngrok_thread.start()

# Wait for ngrok to start (optional)
time.sleep(5)

# Execute your node.js script
!streamlit run app.py

ERROR:pyngrok.process.ngrok:t=2025-06-12T18:58:44+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-06-12T18:58:44+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.187.155.18:8501

  Stopping...
^C


In [ ]:
!streamlit run app.py --server.port 7860




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:7860
  Network URL: http://172.28.0.12:7860
  External URL: http://35.187.155.18:7860

  Stopping...
  Stopping...


In [ ]:
# عرض تطبيق Streamlit داخل Colab
import threading
import streamlit.web.bootstrap
from streamlit.web.server import Server
from streamlit.runtime.scriptrunner import add_script_run_ctx

def run():
    Server.get_current()._is_running = False
    streamlit.web.bootstrap.run('/content/app.py', '', [])

t = threading.Thread(target=run)
add_script_run_ctx(t)
t.start()

2025-06-12 19:01:19.641 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Exception in thread Thread-10 (run):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-6-4252555953>", line 8, in run
AttributeError: type object 'Server' has no attribute 'get_current'


In [ ]:
!pip install localtunnel
!streamlit run app.py
!lt --port 8501

ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.187.155.18:8501

  Stopping...
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/streamlit", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/

In [ ]:
!pip install streamlit localtunnel --quiet


ERROR: Could not find a version that satisfies the requirement localtunnel (from versions: none)
ERROR: No matching distribution found for localtunnel


In [ ]:
import os
import time
import threading

def run_streamlit():
    os.system('streamlit run app.py --server.port 8501')

threading.Thread(target=run_streamlit).start()

# انتظر قليلًا حتى يفتح السيرفر
time.sleep(5)

# افتح نفق خارجي بـ localtunnel
!lt --port 8501


/bin/bash: line 1: lt: command not found


In [ ]:
from pyngrok import ngrok
import threading
import os

# فتح النفق أولاً (بدون توثيق حساب)
public_url = ngrok.connect(8501)
print(f"📨 رابط الواجهة العامة: {public_url}")

# تشغيل Streamlit في الخلفية
def run():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run)
thread.start()


📨 رابط الواجهة العامة: NgrokTunnel: "https://b255-35-187-155-18.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [ ]:
!pip install streamlit langchain langchain-community faiss-cpu sentence-transformers
!streamlit run app.py & npx localtunnel --port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://172.28.0.12:8502
  External URL: http://35.187.155.18:8502

⠙⠹⠸⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://little-rings-shine.loca.lt
  Stopping...
^C


In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = "XXXX"


In [ ]:
from pyngrok import ngrok, conf

# إعداد التوكن هنا
conf.get_default().auth_token = "XXXX"


In [ ]:
# افتح نفق ngrok على البورت 8501
public_url = ngrok.connect(8501)
print("🔗 رابط الوصول:", public_url)

# شغل Streamlit (يفضل أن يكون app.py موجودًا)
!streamlit run app.py &> /dev/null &


🔗 رابط الوصول: NgrokTunnel: "https://beb6-35-187-155-18.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
# 📄 app.py
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

import tempfile
import os

# ↓ استخدام transformers مباشرة
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# إعداد واجهة Streamlit
st.set_page_config(page_title="سؤال وجواب على ملفات PDF", layout="centered")
st.title("🤖📄 اسأل أي شيء عن ملف PDF")

uploaded_file = st.file_uploader("📤 ارفع ملف PDF", type="pdf")

if uploaded_file:
    # حفظ الملف مؤقتاً
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(uploaded_file.read())
        tmp_file_path = tmp_file.name

    # تحميل الملف باستخدام LangChain
    with st.spinner("📚 جاري تحميل الملف..."):
        loader = PyPDFLoader(tmp_file_path)
        documents = loader.load()

        # تقسيم النص
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(documents)

        # استخراج Embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        db = FAISS.from_documents(chunks, embeddings)

        retriever = db.as_retriever()

        # تحميل نموذج flan-t5-base محليًا بدون API
        model_name = "google/flan-t5-base"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=300,
            temperature=0.2
        )

        llm = HuggingFacePipeline(pipeline=pipe)

        qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

        st.success("✅ تم تجهيز الملف! يمكنك الآن طرح الأسئلة.")

    # واجهة لطرح الأسئلة
    question = st.text_input("❓ اكتب سؤالك هنا")

    if question:
        with st.spinner("🤔 جاري التفكير..."):
            result = qa_chain.run(question)
            st.markdown("### ✅ الإجابة:")
            st.write(result)

    # حذف الملف المؤقت بعد الاستخدام
    os.unlink(tmp_file_path)


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

⠙⠹

⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8504
  Network URL: http://172.28.0.12:8504
  External URL: http://35.187.155.18:8504

⠇⠏your url is: https://afraid-seals-post.loca.lt


  Stopping...
^C


In [ ]:
trust_remote_code=True

In [ ]:
!huggingface-cli login --token XXXX

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
The token `read` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `read`


In [ ]:
شغال

/content/app.py



# 📄 app.py
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

import tempfile
import os

# ↓ استخدام transformers مباشرة
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# إعداد واجهة Streamlit
st.set_page_config(page_title="سؤال وجواب على ملفات PDF", layout="centered")
st.title("🤖📄 اسأل أي شيء عن ملف PDF")

uploaded_file = st.file_uploader("📤 ارفع ملف PDF", type="pdf")

if uploaded_file:
    # حفظ الملف مؤقتاً
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp_file:
        tmp_file.write(uploaded_file.read())
        tmp_file_path = tmp_file.name

    # تحميل الملف باستخدام LangChain
    with st.spinner("📚 جاري تحميل الملف..."):
        loader = PyPDFLoader(tmp_file_path)
        documents = loader.load()

        # تقسيم النص
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = splitter.split_documents(documents)

        # استخراج Embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        db = FAISS.from_documents(chunks, embeddings)

        retriever = db.as_retriever()

        # تحميل نموذج flan-t5-base محليًا بدون API
        from transformers import AutoTokenizer, AutoModelForCausalLM

        model_name = "google/gemma-3-1b-it"

        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=300,
            temperature=0.2
        )

        llm = HuggingFacePipeline(pipeline=pipe)

        qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

        st.success("✅ تم تجهيز الملف! يمكنك الآن طرح الأسئلة.")

    # واجهة لطرح الأسئلة
    question = st.text_input("❓ اكتب سؤالك هنا")

    if question:
        with st.spinner("🤔 جاري التفكير..."):
            result = qa_chain.run(question)
            st.markdown("### ✅ الإجابة:")
            st.write(result)

    # حذف الملف المؤقت بعد الاستخدام
    os.unlink(tmp_file_path)


In [ ]:
# افتح نفق ngrok على البورت 8501
public_url = ngrok.connect(8501)
print("🔗 رابط الوصول:", public_url)

# شغل Streamlit (يفضل أن يكون app.py موجودًا)
!streamlit run app.py &> /dev/null &


🔗 رابط الوصول: NgrokTunnel: "https://d7e3-35-187-155-18.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install transformers accelerate torch pdfminer.six


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.3 MB/s eta 0:00:00


### شغال تلخيص جيد

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from pdfminer.high_level import extract_text
import torch
import textwrap

# 1. تحميل النموذج و Tokenizer
model_id = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

# 2. إعداد pipeline للتلخيص
summarizer = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,
)

# 3. استخراج النص من ملف PDF
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# 4. تقسيم النص إلى مقاطع قصيرة لتلخيصها
def chunk_text(text, max_tokens=512):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunk = " ".join(words[i:i+max_tokens])
        chunks.append(chunk)
    return chunks

# 5. تلخيص المقاطع واحدة تلو الأخرى
def summarize_pdf_text(text):
    chunks = chunk_text(text)
    summaries = []

    for i, chunk in enumerate(chunks):
        print(f"\n🔹 تلخيص الجزء {i+1} من {len(chunks)}...")
        prompt = f"لخّص النص التالي بدقة واختصار:\n{chunk}\nالملخص:"
        result = summarizer(prompt, do_sample=False)[0]["generated_text"]
        summary = result.split("الملخص:")[-1].strip()
        summaries.append(summary)

    return "\n\n".join(summaries)

# 6. تنفيذ الكود على ملف PDF
def summarize_pdf(pdf_path):
    print("📄 استخراج النص من PDF...")
    text = extract_text_from_pdf(pdf_path)
    print("✂️ تقسيم النص وتلخيصه...")
    final_summary = summarize_pdf_text(text)
    print("\n✅ الملخص النهائي:\n")
    print(textwrap.fill(final_summary, width=100))

# مثال:
pdf_file = "attention_is_all_you_need.pdf"  # ضع هنا اسم ملفك
summarize_pdf(pdf_file)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


📄 استخراج النص من PDF...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


✂️ تقسيم النص وتلخيصه...

🔹 تلخيص الجزء 1 من 12...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


...


🔹 تلخيص الجزء 10 من 12...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔹 تلخيص الجزء 11 من 12...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



🔹 تلخيص الجزء 12 من 12...

✅ الملخص النهائي:

3 2 0 2 g u A 2 ] L C. s c [ 7 v 2 6 7 3 0. 6 0 7 1 : v i X r a Provided proper attribution is
provided, Google hereby grants permission to reproduce the tables and figures in this paper solely
for use in journalistic or scholarly works. Attention Is All You Need Ashish Vaswani∗ Google Brain
avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research
nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research
llion@google.com Aidan N. Gomez∗ † University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google
Brain lukaszkaiser@google.com Illia Polosukhin∗ ‡ illia.polosukhin@gmail.com  الخلاصة:  تُعدّ سلسلة
من حالات الخفيّات h، كـ f(h, h−1, t)، حيث أن h−1 هو الحالة السابقة و t هو موقع الإدخال. هذه الطبيعة
التسلسلية تمنع التوازي داخل أمثلة التدريب، مما يصبح مهمًا بشكل خاص عند استخدام أطوال التسلسل الأطول،
حيث تقتصر قيود الذاكرة على التجميع عبر الأمثلة. أحدثت العمليات التحسينية 

In [ ]:
# 1. تثبيت المتطلبات
!pip install -q transformers accelerate sentence-transformers faiss-cpu PyPDF2

# 2. استيراد المكتبات
import os
import torch
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import faiss
from IPython.display import display, Markdown

# 3. تحميل المستند (PDF)
pdf_path = "/content/attention_is_all_you_need.pdf"  # ← عدّل اسم الملف لو لزم الأمر
reader = PdfReader(pdf_path)
text = "\n".join([page.extract_text() or "" for page in reader.pages])

# 4. تقطيع النص إلى أجزاء صغيرة
def split_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

chunks = split_text(text)

# 5. تحويل الأجزاء إلى تمثيلات دلالية
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # خفيف ويعمل على المعالج
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

# 6. بناء قاعدة بيانات دلالية باستخدام FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 7. تحميل نموذج لغوي خفيف يعمل على المعالج (يمكن استبداله بنموذج آخر إذا رغبت)
model_id = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, device_map="auto")

# 8. دالة الإجابة على الأسئلة باستخدام البحث الدلالي + LLM
def ask_question(question, top_k=3, max_new_tokens=256):
    q_embedding = embedding_model.encode([question])
    scores, indices = index.search(q_embedding, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = "\n\n".join(relevant_chunks)

    prompt = f"""You are an intelligent assistant that answers questions based on the following information:{context}

Question: {question}
Answer: """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    display(Markdown("### ✅ الجواب:\n" + answer.split("الإجابة:")[-1].strip()))

# 9. تجربة السؤال
ask_question("what is attention؟")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### ✅ الجواب:
You are an intelligent assistant that answers questions based on the following information:

position, ensures that the predictions for position ican depend only on the known outputs at positions less than i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3 Scaled Dot-Product Attention Multi-Head Attention Figure 2: (left) Scaled Dot-Product Attention. (right) Multi-Head Attention consists of several attention layers running in parallel. of the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key. 3.2.1 Scaled Dot-Product Attention We call our particular attention "Scaled Dot-Product Attention" (Figure 2). The input consists of queries and keys of dimension dk, and values of dimension dv. We compute the dot products of the query with all keys, divide each by√dk, and apply a softmax function to obtain the weights on the values. In practice, we compute the attention function on a set of queries simultaneously, packed together into a matrix Q. The keys and values are also packed together into matrices KandV. We compute the matrix of outputs as: Attention( Q, K, V ) = softmax(QKT √dk)V (1) The two most commonly used attention functions are additive attention [ 2], and dot-product (multi- plicative) attention. Dot-product attention is identical to our algorithm, except for the scaling factor of1√dk. Additive attention computes the compatibility function using a feed-forward network with a single hidden layer. While the two are similar in theoretical complexity, dot-product attention is much faster and more space-efficient in practice, since it can be implemented using highly optimized matrix multiplication code. While for small values of dkthe two mechanisms perform similarly, additive attention outperforms dot product attention without scaling for larger values

ByteNet. This makes it more difficult to learn dependencies between distant positions [ 12]. In the Transformer this is reduced to a constant number of operations, albeit at the cost of reduced effective resolution due to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as described in section 3.2. Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 27, 28, 22]. End-to-end memory networks are based on a recurrent attention mechanism instead of sequence- aligned recurrence and have been shown to perform well on simple-language question answering and language modeling tasks [34]. To the best of our knowledge, however, the Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate self-attention and discuss its advantages over models such as [17, 18] and [9]. 3 Model Architecture Most competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,35]. Here, the encoder maps an input sequence of symbol representations (x1, ..., x n)to a sequence of continuous representations z= (z1, ..., z n). Given z, the decoder then generates an output sequence (y1, ..., y m)of symbols one element at a time. At each step the model is auto-regressive [10], consuming the previously generated symbols as additional input when generating the next. 2 Figure 1: The Transformer - model architecture. The Transformer follows this overall architecture using stacked self-attention and point-wise, fully connected layers for both the encoder and decoder, shown in the left and right

parameter matrices WQ i∈Rdmodel×dk,WK i∈Rdmodel×dk,WV i∈Rdmodel×dv andWO∈Rhdv×dmodel. In this work we employ h= 8 parallel attention layers, or heads. For each of these we use dk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost is similar to that of single-head attention with full dimensionality. 3.2.3 Applications of Attention in our Model The Transformer uses multi-head attention in three different ways: •In "encoder-decoder attention" layers, the queries come from the previous decoder layer, and the memory keys and values come from the output of the encoder. This allows every position in the decoder to attend over all positions in the input sequence. This mimics the typical encoder-decoder attention mechanisms in sequence-to-sequence models such as [38, 2, 9]. •The encoder contains self-attention layers. In a self-attention layer all of the keys, values and queries come from the same place, in this case, the output of the previous layer in the encoder. Each position in the encoder can attend to all positions in the previous layer of the encoder. •Similarly, self-attention layers in the decoder allow each position in the decoder to attend to all positions in the decoder up to and including that position. We need to prevent leftward information flow in the decoder to preserve the auto-regressive property. We implement this inside of scaled dot-product attention by masking out (setting to −∞) all values in the input of the softmax which correspond to illegal connections. See Figure 2. 3.3 Position-wise Feed-Forward Networks In addition to attention sub-layers, each of the layers in our encoder and decoder contains a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between. FFN( x) = max(0 , xW 1+b1)W2+b2 (2) While the linear transformations

Question: what is attention؟
Answer: 
Attention is a mechanism that maps a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors.

Question: what is the purpose of the scaled dot-product attention?
Answer:
The purpose of the scaled dot-product attention is to compute the attention weights on the values, where the weight assigned to each value is computed by a compatibility function of the query with the corresponding key.

Question: what is the role of the scaling factor in the scaled dot-product attention?
Answer:
The scaling factor is 1√dk, which is used to prevent the attention weights from becoming too large.

Question: what is the main advantage of the Transformer over models such as [17, 18] and [9]?
Answer:
The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence- aligned RNNs or convolution.

Question: what is the role of the masked out values in the scaled dot-product attention?
Answer:
The masked out values in the scaled dot-product attention are used to prevent leftward information flow in the decoder to preserve the auto-regressive property.

In [ ]:
# 1. تثبيت المتطلبات
!pip install -q transformers accelerate sentence-transformers faiss-cpu PyPDF2

# 2. استيراد المكتبات
import os
import torch
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import faiss
from IPython.display import display, Markdown

# 3. تحميل المستند (PDF)
pdf_path = "/content/attention_is_all_you_need.pdf"  # ← عدّل اسم الملف لو لزم الأمر
reader = PdfReader(pdf_path)
text = "\n".join([page.extract_text() or "" for page in reader.pages])

# 4. تقطيع النص إلى أجزاء صغيرة
def split_text(text, chunk_size=300, overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+chunk_size]
        chunks.append(" ".join(chunk))
        i += chunk_size - overlap
    return chunks

chunks = split_text(text)

# 5. تحويل الأجزاء إلى تمثيلات دلالية
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # خفيف ويعمل على المعالج
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

# 6. بناء قاعدة بيانات دلالية باستخدام FAISS
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 7. تحميل نموذج لغوي خفيف يعمل على المعالج (يمكن استبداله بنموذج آخر إذا رغبت)
model_id = "google/gemma-3-1b-it"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, device_map="auto")

# 8. دالة الإجابة على الأسئلة باستخدام البحث الدلالي + LLM
def ask_question(question, top_k=3, max_new_tokens=256):
    q_embedding = embedding_model.encode([question])
    scores, indices = index.search(q_embedding, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = "\n\n".join(relevant_chunks)

    prompt = f"""You are an intelligent assistant that answers questions based on the following information:{context}

Question: {question}
Answer: """

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cpu")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False, temperature=0.7)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    display(Markdown("### ✅ الجواب:\n" + answer.split("الإجابة:")[-1].strip()))

# 9. تجربة السؤال
ask_question("what is attention؟")


In [ ]:
!pip install -q sentence-transformers langchain transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

# Load the semantic search model and answer generation model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # Lightweight and CPU-friendly
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1) # CPU-friendly

# Load the document as text (you can change it here)
with open("document.txt", "r", encoding="utf-8") as f:
full_text = f.read()

# Split the document into small paragraphs for semantic search
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

# Generate semantic numerical representations of paragraphs
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# Function for answering questions using semantic search
def answer_question(question):
question_embedding = embedding_model.encode(question, convert_to_tensor=True)

# Calculate the similarity of the question to all paragraphs in the document
similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
best_idx = torch.argmax(similarities).item()
best_chunk = chunks[best_idx]

# Generate a smart answer based on the best paragraph
prompt = f "Extract a smart answer for the following question from the text:\n\nText: {best_chunk}\n\nQuestion: {question}\nAnswer:"
response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
return response.strip()

# Example: Using the function with a question
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))

IndentationError: expected an indented block after 'with' statement on line 11 (<ipython-input-3-51066654>, line 12)

In [ ]:
model_id = 'mobiuslabsgmbh/Llama-2-7b-chat-hf-4bit_g64-HQQ'

from hqq.engine.hf import HQQModelForCausalLM, AutoTokenizer
import transformers
from threading import Thread
import torch
from sys import stdout

def print_flush(data):
    stdout.write("\r" + data)
    stdout.flush()

# Ensure CUDA is available
assert torch.cuda.is_available(), "CUDA must be available!"
device = torch.device('cuda')
print(f"Using device: {device}")

torch.set_default_tensor_type('torch.cuda.FloatTensor')

# Load model and tokenizer
def init_model():
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = HQQModelForCausalLM.from_quantized(model_id)
    model = model.to(device)
    model.eval()

    # Check all model parameters are on CUDA
    for param in model.parameters():
        assert param.device.type == 'cuda'

    return model, tokenizer

model, tokenizer = init_model()

# Build input chat sequence
def process_conversation(chat):
    system_prompt = chat['system_prompt']
    chat_history = chat['chat_history']
    message = chat['message']

    conversation = []
    if system_prompt:
        conversation.append({"role": "system", "content": system_prompt})
    for user, assistant in chat_history:
        conversation.extend([
            {"role": "user", "content": user},
            {"role": "assistant", "content": assistant}
        ])
    conversation.append({"role": "user", "content": message})

    with torch.cuda.device(device):
        input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt").to(device)
        attention_mask = torch.ones_like(input_ids, dtype=torch.long, device=device)

        inputs = {
            "input_ids": input_ids,
            "attention_mask": attention_mask
        }

        return inputs

@torch.inference_mode()
def chat_processor(chat, max_new_tokens=100, do_sample=True):
    try:
        tokenizer.use_default_system_prompt = False
        streamer = transformers.TextIteratorStreamer(
            tokenizer,
            timeout=10.0,
            skip_prompt=True,
            skip_special_tokens=True
        )

        with torch.cuda.device(device):
            inputs = process_conversation(chat)

            generate_kwargs = {
                **inputs,
                "streamer": streamer,
                "max_new_tokens": max_new_tokens,
                "do_sample": do_sample,
                "top_p": 0.90,
                "top_k": 50,
                "temperature": 0.6,
                "num_beams": 1,
                "repetition_penalty": 1.2,
                "pad_token_id": tokenizer.pad_token_id,
                "eos_token_id": tokenizer.eos_token_id,
                "bos_token_id": tokenizer.bos_token_id
            }

            t = Thread(target=lambda: model.generate(**generate_kwargs))
            t.start()

            outputs = []
            for text in streamer:
                outputs.append(text)
                print(text, end="")
                stdout.flush()

            return outputs

    except Exception as e:
        print(f"Error in chat_processor: {str(e)}")
        print("\nCUDA Memory Summary:")
        print(torch.cuda.memory_summary())
        raise

torch.cuda.empty_cache()

# Sample chat input
try:
    with torch.cuda.device(device):
        chat_data = {
            'system_prompt': "You are a helpful assistant.",
            'chat_history': [],
            'message': "What is the purpose of attention in transformer models?"
        }
        outputs = chat_processor(chat_data, max_new_tokens=128, do_sample=False)
except Exception as e:
    print(f"\nError during execution: {str(e)}")
    print(f"CUDA memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    print(f"CUDA memory cached: {torch.cuda.memory_reserved()/1e9:.2f} GB")
    raise


ModuleNotFoundError: No module named 'hqq'

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

# تحميل نموذج البحث الدلالي ونموذج توليد الإجابات
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # خفيف ويعمل على CPU
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)  # على CPU

# تحميل المستند كنص (يمكنك تغييره هنا)
with open("document.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم المستند إلى فقرات صغيرة قابلة للبحث الدلالي
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

# إنشاء التمثيلات العددية الدلالية للفقرات
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة الإجابة على الأسئلة باستخدام البحث الدلالي
def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب تشابه السؤال مع جميع فقرات المستند
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # توليد إجابة ذكية بناءً على أفضل فقرة
    prompt = fExtract a clever answer to the following question from the text:\n\nText: {best_chunk}\n\nQuestion: {question}\nالإجابة:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال: استخدام الدالة مع سؤال
question = "ما هو الفرق بين attention و self-attention؟"
print("السؤال:", question)
print("الإجابة:", answer_question(question))


In [ ]:
!pip install pypdf2 pdfplumber pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 29.0 MB/s eta 0:00:00


In [ ]:
import pdfplumber

def pdf_to_text_plumber(pdf_path, output_txt_path):
    """
    تحويل ملف PDF إلى ملف نصي باستخدام pdfplumber (أكثر دقة)

    :param pdf_path: مسار ملف PDF المدخل
    :param output_txt_path: مسار ملف النص المخرَج
    """
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()

        with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    print(f"تم تحويل PDF إلى نص بنجاح وحفظه في: {output_txt_path}")

# مثال للاستخدام
pdf_to_text_plumber('input.pdf', 'output.txt')

In [ ]:
import PyPDF2

def pdf_to_text(pdf_path, output_txt_path):
    """
    تحويل ملف PDF إلى ملف نصي باستخدام PyPDF2

    :param pdf_path: مسار ملف PDF المدخل
    :param output_txt_path: مسار ملف النص المخرَج
    """
    with open(pdf_path, 'rb') as pdf_file:
        # إنشاء قارئ PDF
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        # استخراج النص من كل صفحة
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

        # حفظ النص في ملف
        with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)

    print(f"تم تحويل PDF إلى نص بنجاح وحفظه في: {output_txt_path}")

# مثال للاستخدام
pdf_to_text('input.pdf', 'output.txt')

In [ ]:
/content/attention_is_all_you_need.pdf

In [ ]:
from pdfminer.high_level import extract_text

def pdf_to_text_miner(pdf_path, output_txt_path):
    """
    تحويل ملف PDF إلى ملف نصي باستخدام pdfminer.six (يدعم نصوص معقدة)

    :param pdf_path: مسار ملف PDF المدخل
    :param output_txt_path: مسار ملف النص المخرَج
    """
    text = extract_text(pdf_path)

    with open(output_txt_path, 'w', encoding='utf-8') as txt_file:
        txt_file.write(text)

    print(f"تم تحويل PDF إلى نص بنجاح وحفظه في: {output_txt_path}")

# مثال للاستخدام
pdf_to_text_miner('attention_is_all_you_need.pdf', 'output.txt')

تم تحويل PDF إلى نص بنجاح وحفظه في: output.txt


In [ ]:
google/flan-t5-base

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

# تحميل نموذج البحث الدلالي ونموذج توليد الإجابات
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # خفيف ويعمل على CPU
qa_pipeline = pipeline("text2text-generation", model="google/gemma-3-1b-it", device=-1)  # على CPU

# تحميل المستند كنص (يمكنك تغييره هنا)
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم المستند إلى فقرات صغيرة قابلة للبحث الدلالي
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

# إنشاء التمثيلات العددية الدلالية للفقرات
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة الإجابة على الأسئلة باستخدام البحث الدلالي
def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب تشابه السؤال مع جميع فقرات المستند
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # توليد إجابة ذكية بناءً على أفضل فقرة
    prompt = fExtract a clever answer to the following question from the text:\n\nText: {best_chunk}\n\nQuestion: {question}\nالإجابة:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال: استخدام الدالة مع سؤال
question = "What is the difference between attention and self-attention?"
print("السؤال:", question)
print("الإجابة:", answer_question(question))


SyntaxError: invalid syntax (<ipython-input-1-3592452524>, line 31)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

# تحميل نموذج البحث الدلالي ونموذج توليد الإجابات
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # خفيف ويعمل على CPU
qa_pipeline = pipeline("text2text-generation", model="google/gemma-3-1b-it", device=-1)  # على CPU

# تحميل المستند كنص (يمكنك تغييره هنا)
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم المستند إلى فقرات صغيرة قابلة للبحث الدلالي
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

# إنشاء التمثيلات العددية الدلالية للفقرات
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة الإجابة على الأسئلة باستخدام البحث الدلالي
def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب تشابه السؤال مع جميع فقرات المستند
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # توليد إجابة ذكية بناءً على أفضل فقرة
    prompt = f"Extract a smart answer to the following question from the text:\n\nText: {best_chunk}\n\nQuestion: {question}\nAnswer:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال: استخدام الدالة مع سؤال
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))



KeyboardInterrupt



In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

# تحميل نموذج البحث الدلالي ونموذج توليد الإجابات
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # خفيف ويعمل على CPU
qa_pipeline = pipeline("text2text-generation", model="google/gemma-3-1b-it", device=-1)  # على CPU

# تحميل المستند كنص (يمكنك تغييره هنا)
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم المستند إلى فقرات صغيرة قابلة للبحث الدلالي
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

# إنشاء التمثيلات العددية الدلالية للفقرات
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة الإجابة على الأسئلة باستخدام البحث الدلالي
def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب تشابه السؤال مع جميع فقرات المستند
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # توليد إجابة ذكية بناءً على أفضل فقرة
    prompt = f"Extract a smart answer to the following question from the text:\n\nText: {best_chunk}\n\nQuestion: {question}\nAnswer:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال: استخدام الدالة مع سؤال
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
The model 'Gemma3ForCausalLM' is not supported for text2text-generation. Supported models are ['PeftModelForSeq2SeqLM', 'BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'GraniteSpeechForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForCondit

Question: What is the difference between attention and self-attention?
Answer: Extract a smart answer to the following question from the text:

Text: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].

Question: What is the difference between attention and self-attention?
Answer:
Self-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.
[4, 27, 28, 22]


 ما المشكلة الفعلية؟
التحذير التالي هو لبّ المشكلة:

The model 'Gemma3ForCausalLM' is not supported for text2text-generation.

هذا يعني:

أنك تستخدم نموذج google/gemma-3-1b-it الذي لا يعمل مع نوع المهام "text2text-generation".

النموذج gemma-3-1b-it هو causal language model (أي يستخدم لـ text generation بطريقة text -> continuation) وليس مناسبًا لطريقة text2text-generation.

✅ الحل: استخدام pipeline المناسب
بدلاً من "text2text-generation" استخدم "text-generation":

python
Copy
Edit
qa_pipeline = pipeline("text-generation", model="google/gemma-3-1b-it", device=-1)
ثم غيّر السطر داخل answer_question ليأخذ فقط الناتج الأول من النص (لأن text-generation يُكمل النص بدلًا من إنتاج إجابة مباشرة):

python
Copy
Edit
response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
💡 ملاحظة: يجب أن يكون prompt مكتوبًا بأسلوب يُناسب الإكمال، مثل:

python
Copy
Edit
prompt = f"Question: {question}\nText: {best_chunk}\nAnswer:"
💡 اقتراح: تحسين المخرجات
إذا كنت تريد أسلوبًا أقرب لنمط سؤال-جواب، فاستخدم نموذجًا مثل google/flan-t5-base أو google/flan-t5-large، وهما يعملان بكفاءة على "text2text-generation".

مثال:

python
Copy
Edit
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)
🔧 مشكلة التقسيم:
ظهور التحذيرات مثل:

arduino
Copy
Edit
Created a chunk of size 1140, which is longer than the specified 500
يعني أن CharacterTextSplitter فشل في تقسيم بعض النصوص الطويلة. يمكنك استخدام RecursiveCharacterTextSplitter لتقسيم أفضل حسب الجمل والفقرات.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
qa_pipeline = pipeline("text-generation", model="google/gemma-3-1b-it", device=-1)

with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    prompt = f"Question: {question}\nText: {best_chunk}\nAnswer:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Question: What is the difference between attention and self-attention?
Answer: Question: What is the difference between attention and self-attention?
Text: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 27, 28, 22].
Answer:
The difference between attention and self-attention is that self-attention is a
specific type of attention mechanism.
The answer is:
Self-attention is a specific type of attention mechanism.
The answer is:
Self-attention is a specific type of attention mechanism.
The answer is:
Self-attention is a specific type of attention mechanism.
The answer is:
Self-attention is a specific type of attention mechanism.
The answer is:
Self-attention is

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
#qa_pipeline = pipeline("text-generation", model="google/gemma-3-1b-it", device=-1)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)


with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    prompt = f"Question: {question}\nText: {best_chunk}\nAnswer:"
    response = qa_pipeline(prompt, max_new_tokens=128, do_sample=False)[0]['generated_text']
    return response.strip()

question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Question: What is the difference between attention and self-attention?
Answer: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence. Self-attention has been used successfully in a variety of tasks including reading comprehension, abstractive summarization, textual entailment and learning task-independent sentence representations [4, 27, 28, 22].


In [ ]:

from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import CharacterTextSplitter

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
#qa_pipeline = pipeline("text-generation", model="google/gemma-3-1b-it", device=-1)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)


with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(full_text)

chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)















def answer_question(question):
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    prompt = (
        f"Answer the question based on the text.\n\n"
        f"Text:\n{best_chunk}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    response = qa_pipeline(prompt, max_new_tokens=256, do_sample=False)[0]['generated_text']
    return response.strip()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تحميل نموذج التضمين
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# تحميل نموذج الإجابة (نموذج توليد النصوص)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)

# قراءة النص الكامل من الملف
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم النص باستخدام RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_text(full_text)

# حساب تمثيلات التضمين لجميع القطع
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# تعريف دالة الإجابة على الأسئلة
def answer_question(question):
    # حساب تمثيل التضمين للسؤال
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب التشابه بين السؤال وجميع القطع
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]

    # اختيار القطعة الأكثر تشابهًا
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # إنشاء الموجه (prompt) للنموذج
    prompt = (
        f"Answer the question based on the text.\n\n"
        f"Text:\n{best_chunk}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    # توليد الإجابة باستخدام النموذج
    response = qa_pipeline(prompt, max_new_tokens=256, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال على الاستخدام
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Question: What is the difference between attention and self-attention?
Answer: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تحميل نموذج التضمين
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# تحميل نموذج الإجابة (نموذج توليد النصوص)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)

# قراءة النص الكامل من الملف
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم النص باستخدام RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_text(full_text)

# حساب تمثيلات التضمين لجميع القطع
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# تعريف دالة الإجابة على الأسئلة
def answer_question(question):
    # حساب تمثيل التضمين للسؤال
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب التشابه بين السؤال وجميع القطع
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]

    # اختيار القطعة الأكثر تشابهًا
    best_idx = torch.argmax(similarities).item()
    best_chunk = chunks[best_idx]

    # إنشاء الموجه (prompt) للنموذج
    prompt = (
        f"Answer the question based on the text.\n\n"
        f"Text:\n{best_chunk}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    # توليد الإجابة باستخدام النموذج
    response = qa_pipeline(prompt, max_new_tokens=1024, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال على الاستخدام
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


### شغال

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تحميل نموذج التضمين
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# تحميل نموذج الإجابة (نموذج توليد النصوص)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=-1)

# قراءة النص الكامل من الملف
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم النص باستخدام RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_text(full_text)

# حساب تمثيلات التضمين لجميع القطع
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# تعريف دالة الإجابة على الأسئلة
def answer_question(question, top_k=5):
    # حساب تمثيل التضمين للسؤال
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب التشابه بين السؤال وجميع القطع
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]

    # اختيار أفضل k قطع
    top_indices = torch.topk(similarities, k=top_k).indices.tolist()
    combined_text = "\n\n".join([chunks[i] for i in top_indices])

    # إنشاء الموجه (prompt) للنموذج
    prompt = (
        f"Answer the question based on the following text.\n\n"
        f"Text:\n{combined_text}\n\n"
        f"Question: {question}\n\n"
        f"Answer:"
    )

    # توليد الإجابة باستخدام النموذج
    response = qa_pipeline(prompt, max_new_tokens=1024, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال على الاستخدام
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Question: What is the difference between attention and self-attention?
Answer: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence


شغال

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# تحديد الجهاز تلقائيًا (CUDA إن توفر، وإلا CPU)
device = 0 if torch.cuda.is_available() else -1
print(f"Device set to: {'cuda' if device == 0 else 'cpu'}")

# تحميل نموذج التضمين
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# تحميل نموذج الإجابة على الأسئلة
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-large", device=device)

# قراءة النص الكامل من الملف
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم النص إلى مقاطع باستخدام RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_text(full_text)

# حساب تمثيلات التضمين لجميع المقاطع
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة للإجابة على الأسئلة
def answer_question(question, top_k=3):
    # تمثيل تضمين السؤال
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب التشابه مع كل المقاطع
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]

    # الحصول على أفضل k مقاطع أكثر تشابهًا
    top_indices = torch.topk(similarities, k=top_k).indices.tolist()
    combined_text = "\n\n".join([chunks[i] for i in top_indices])

    # إنشاء الموجه (prompt) المحسن
    prompt = (
        "Use the following context to answer the question clearly and concisely.\n\n"
        f"Context:\n{combined_text}\n\n"
        f"Question: {question}\n\n"
        "Answer:"
    )

    # توليد الإجابة
    response = qa_pipeline(prompt, max_new_tokens=256, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال على الاستخدام
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


Device set to: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


Question: What is the difference between attention and self-attention?
Answer: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence


### شغال

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# اختيار الجهاز تلقائيًا (CUDA إن وُجد، وإلا CPU)
device = 0 if torch.cuda.is_available() else -1
print("Using device:", "cuda" if device == 0 else "cpu")

# تحميل نموذج التضمين
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# تحميل نموذج الإجابة (Flan-T5 base مناسب للفهم والتعليمات)
qa_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", device=device)

# قراءة النص الكامل من الملف
with open("output.txt", "r", encoding="utf-8") as f:
    full_text = f.read()

# تقسيم النص إلى مقاطع باستخدام RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", "!", "?", " "]
)
chunks = text_splitter.split_text(full_text)

# تحويل المقاطع إلى تمثيلات عددية (embeddings)
chunk_embeddings = embedding_model.encode(chunks, convert_to_tensor=True)

# دالة الإجابة على الأسئلة
def answer_question(question, top_k=3):
    # تضمين السؤال
    question_embedding = embedding_model.encode(question, convert_to_tensor=True)

    # حساب التشابه بين السؤال وكل المقاطع
    similarities = util.pytorch_cos_sim(question_embedding, chunk_embeddings)[0]

    # اختيار أفضل k مقاطع
    top_indices = torch.topk(similarities, k=top_k).indices.tolist()
    combined_text = "\n\n".join([chunks[i] for i in top_indices])

    # موجه مطور (prompt) للحصول على إجابة تحليلية
    prompt = (
        f"You are an expert in machine learning.\n\n"
        f"Based on the following text, compare and contrast attention and self-attention clearly and analytically.\n\n"
        f"Text:\n{combined_text}\n\n"
        f"Question: What is the difference between attention and self-attention?\n\n"
        f"Answer:"
    )

    # توليد الإجابة
    response = qa_pipeline(prompt, max_new_tokens=300, do_sample=False)[0]['generated_text']
    return response.strip()

# مثال على الاستخدام
question = "What is the difference between attention and self-attention?"
print("Question:", question)
print("Answer:", answer_question(question))


Using device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Question: What is the difference between attention and self-attention?
Answer: Self-attention, sometimes called intra-attention is an attention mechanism relating different positions of a single sequence in order to compute a representation of the sequence.
